# Quantization
This notebook acts as an example of how to use the quantization techniques.

## Setup
* Import the necessary packages.
* Load a model.
* Load a dataset.
* Analyze performance of the model prior to quantization.

In [3]:
import torch
import importlib
import inspect
import sys

if '../' not in sys.path:
    sys.path.append('../')


In [4]:
model_state = "../models/mnist.pt"
model_class = "models.mnist"


# Import the module class
module = importlib.import_module(model_class)

# Get all classes in the module
classes = [
    obj[1] for obj in inspect.getmembers(module, inspect.isclass)
]

# Import the classes that are Modules
for cls in classes:
    if issubclass(cls, torch.nn.Module):
        # Add the class to this package's variables
        globals()[cls.__name__] = cls

model = torch.load(model_state, map_location=torch.device('cpu'))

# Print the model's parameters to confirm the model is loaded correctly
for i in model.named_parameters():
    print(i[0])

conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
